In [3]:
from collections import namedtuple

In [156]:
class System:
    _k1 = 1.34
    _k2 = 1.6E9
    _k3 = 8000
    _k4 = 4E7
    _k5 = 0.5
    _f = 1.45
    _a = 0.06
    F = namedtuple('F' , 'x y z') 
    
    def F_init(self,x,y,z):
        #dX/dt
        self.F.x = self._k1*self._a*y-self._k2*x*y+self._k3*self._a*x-2*self._k4*x**2
        #dY/dt
        self.F.y = -self._k1*self._a*y-self._k2*x*y+self._f*self._k5*z
        #dZ/dt
        self.F.z = self._k3*self._a*x-self._k5*z
        return list([self.F.x ,self.F.y,self.F.z])
        '''
        self.F.x = x**2+y**2+z**2-1
        self.F.y = 2*x**2+y**2-4*z
        self.F.z = 3*x**2-4*y+z**2
        '''
    def F_derivative_init(self,x,y,z):
        F_derivative = [
            [-self._k2*y+self._k3*self._a-4*self._k4*x,self._k1*self._a-self._k2*x, 0],
            [-self._k2*y, -self._k1*self._a-self._k2*x, self._f*self._k5],
            [self._a*self._k3, 0, -self._k5]
            #[2*x, 2*y, 2*z],
            #[4*x, 2*y, -4],
            #[6*x, -4, 2*z]
              ]
        return F_derivative
                    
    def Gauss(self, matrix, dim):
        x = [0 for i in range(dim)]
        for j in range(dim):
            ind = j
            maximum = matrix[j][j]
            for i in range(j+1,dim):
                if (matrix[i][j] > maximum):
                    maximum = matrix[i][j]
                    ind = i
            matrix[ind],matrix[j]=matrix[j],matrix[ind]
            
            for i in range (j+1,dim):
                d = -matrix[i][j]/matrix[j][j]
                for j1 in range(dim+1):
                    matrix[i][j1]+=matrix[j][j1]*d
                    
        for i in reversed(range(dim)):
            summ = matrix[i][dim]
            for j in reversed(range(dim)):
                summ-=matrix[i][j]*x[j]
            x[i] = summ/matrix[i][i]
        return x
    
    def Sum(self, x,y):
        return list(map(lambda a, b: a + b, x, y))
    
    def Difference(self, x,y):
        return list(map(lambda a, b: a - b, x, y))
    
    def PrintMatr(self, matr):
        dim = len(matr)
        print()
        for i in range(dim):
                print(matr[i])
        print()
    
    def GetDeterm(self, matr, dim):
        det = 0
        if (dim<1):
            print("Determinant cannot being counted!")
        elif (dim==1):
            det = matr[0][0]
        elif (dim == 2):
            det = matr[0][0] * matr[1][1] - matr[0][1] * matr[1][0]
        else:
            minus = 1
            newmatr = [0]*(dim-1)
            for ni in range(dim-1):
                newmatr[ni]=[0]*dim
            for ni in range(dim):
                self.GetMatrLessDim(matr, newmatr, ni, 0, dim)
                det = det + minus * matr[ni][0] * self.GetDeterm(newmatr, dim - 1)
                minus = -minus
        return det
    
    def GetMatrLessDim(self, matr, newmatr, i, j, dim):
        biasI=0
        for ni in range(dim-1):
            if (ni==i):
                biasI=1
            biasJ=0
            for nj in range(dim-1):
                if (nj==j):
                    biasJ=1
                newmatr[ni][nj]=matr[ni+biasI][nj+biasJ]
        return newmatr
    
    def TranspMtx(self, matr):
        n = len(matr)
        tmatr = [0]*n
        for i in range(n):
            tmatr[i] = [0]*n
        for i in range(n):
            for j in range(n):
                tmatr[j][i]=matr[i][j]
        return tmatr
        
    def GetInverseMatrix(self, matr):
        n = len(matr)
        
        back_matr = [0]*n
        for i in range(n):
            back_matr[i] = [0]*n
            
        det = self.GetDeterm(matr, n)
        if(det):
            for i in range(n):
                for j in range(n):
                    m = n-1
                    temp_matr = [0]*m
                    for k in range(m):
                        temp_matr[k] = [0]*m
                    self.GetMatrLessDim(matr,temp_matr,i,j,n)
                    back_matr[i][j]=(-1.0)**(i+j)*self.GetDeterm(temp_matr, m) / det
            back_matr = self.TranspMtx(back_matr)
        else:
            print("Т.к. определитель матрицы = 0,то матрица вырожденная и обратной не имеет!!!")
            return 
        return back_matr
    
    def MatrixMultiplyVector(self,matr, vector):
        answer = [0]*len(vector)
        for i in range(len(matr)):
            for j in range(len(matr[0])):                
                answer[i]+=matr[i][j]*vector[j]
        return answer
    
    def Newthon(self,x,y,z): 
        x0=[x,y,z]
        f_x0 = list(self.F_init(x,y,z))
        w = self.F_derivative_init(x,y,z)
        if (self.GetDeterm(w, len(w)) == 0):
            return -1
        w_back = self.GetInverseMatrix(w)
        x1 = self.Difference(x0,self.MatrixMultiplyVector(w_back,f_x0)) 
        
        for i in range (1000):
            x0=x1
            f_x0 = list(self.F_init(x0[0],x0[1],x0[2]))
            w = self.F_derivative_init(x0[0],x0[1],x0[2])
            if (self.GetDeterm(w, len(w)) == 0):
                return -1
            w_back = self.GetInverseMatrix(w)
            x1 = self.Difference(x0,self.MatrixMultiplyVector(w_back,f_x0)) 
        
        return x1
    
    def __init__(self,x,y,z):
        self.F_init(x,y,z)
        self.F_derivative_init(x,y,z)

In [157]:
d = System(0.5,0.5,0.5)
#d.Newthon(1,2,3)
d.Newthon(0.5,0.5,0.5)

[2.735505274795964e-10, 3.6749316123681304e-07, 2.6260850638041253e-07]